In [1]:
# grammars (trained by neural network) 
# --> input
# --> CKY 
# --> CKY Table (Parsing Result) 
# --> Tree Representation
# --> Statistical Analysis 

In [1]:
import numpy as np
from parse_statistics.parsing_statistics import ParseTreeStatistics
from backtracker import to_parse_tree, to_grammar_derivation_sequence
import sys
sys.path.append("parsing_statistics")
from eeg_lang import MatrixAndBitmapBasedExplicitPreterminateCNFPCFG

import tvm
import tvm.testing
from tvm import te # tensor expression
import numpy as np

from tvm.script import tir as T
from tvm import meta_schedule as ms
from tvm.script.parser.tir import evaluate

## Prepare grammars

In [2]:
cnt_N = 145
cnt_P = 45
cnt_T = 4
cnt_NT = cnt_N - cnt_T
r = 70
grammar = MatrixAndBitmapBasedExplicitPreterminateCNFPCFG(N=cnt_N, P=cnt_P, T=cnt_T, low_rank_decomposition=70)
[grammar.grammars[key].shape for key in grammar.grammars]

[(1, 100), (100, 70), (145, 70), (145, 70), (45, 4)]

In [3]:
n = 23
class Module_FastCKY_PCFG:
    @T.prim_func
    def main(unary: T.Buffer((n, cnt_T), "float32"),
             root:  T.Buffer((cnt_NT), "float32"),
             H:  T.Buffer((cnt_NT, r), "float32"),
             L:  T.Buffer((cnt_N, r), "float32"),
             R:  T.Buffer((cnt_N, r), "float32"),

             ):
        
        

In [ ]:
@tvm.script.ir_module
class CKYModule_SimplePCFG:
  @T.prim_func
  def main(sequence: T.Buffer((SequenceSize), "int8"),
           L: T.buffer((N, N)),
           R: T.buffer((N, N)),
           P: T.buffer((N, Te)),
           len: T.int16,
           out_cky_table: T.buffer((SequenceSize, SequenceSize, N)),
           aux_beta: T.buffer((SequenceSize, SequenceSize, N)),
           aux_zeta: T.buffer((SequenceSize, SequenceSize, N))
    ):
    T.func_attr({"global_symbol": "main", "tir.noalias": True})
    
    for i in T.grid(len):
      with T.block("block_init_cky"):
        with T.init():
          for j in range(0, N):
            out_cky_table[i, i, j] = P[j, sequence[i]]
    
    for i in T.grid(len):
      for axis_left_symbol in T.grid(N):
        for axis_right_symbol in T.grid(N):
          with T.block("block_init_aux_array"):
            with T.init():
              aux_beta[i, i, axis_left_symbol] += L[axis_left_symbol, axis_right_symbol] * out_cky_table[i, i, axis_right_symbol]
              aux_zeta[i, i, axis_left_symbol] += R[axis_left_symbol, axis_right_symbol] * out_cky_table[i, i, axis_right_symbol]
            
    for t in range(1, len):
      for i in range(0, len - t):
        cell_cor_i = i + t
        cell_cor_j = i
        for symbol_A in T.grid(N):
          for k in range(cell_cor_i, cell_cor_j):
            for symbol_B, symbol_C in T.grid(N, N):
              with T.block("block_cky_main"):
                with T.init():
                  out_cky_table[cell_cor_i, cell_cor_j, symbol_A] += aux_beta[cell_cor_i, k, symbol_B] * aux_beta[k, cell_cor_j, symbol_C]
